In [1]:
import os, sys, re, datetime, random, gzip, json, copy
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from itertools import accumulate
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

from time import time
from math import ceil
from multiprocessing import Pool
from sentence_transformers import SentenceTransformer, models, losses, InputExample

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything

from torch_geometric.nn import Sequential, HeteroConv, GINConv, GCNConv, SAGEConv, GATConv

PROJ_PATH = Path(os.path.join(re.sub("/BERT_ABSA.*$", '', os.getcwd()), 'BERT_ABSA'))
print(f'PROJ_PATH={PROJ_PATH}')
sys.path.insert(1, str(PROJ_PATH))
sys.path.insert(1, str(PROJ_PATH/'src'))
import utils
from utils import *
from dataset import DataModule
from attention import *

/tmp/ipykernel_235065/2553875025.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


PROJ_PATH=/home/hoang/github/BERT_ABSA
PROJ_PATH=/home/hoang/github/BERT_ABSA


In [2]:
# glob.glob('../model/restaurants/*.ckpt')

## Dataset

In [3]:
data = pd.read_pickle('../dataset/preprocessed_data/Restaurants_Trial_data.pkl')
sample = data[0]
print(sample.keys())
print(sample)

dict_keys(['id', 'text', 'term', 'from', 'to', 'source_dep', 'target_dep', 'edge_type', 'label'])
{'id': '813', 'text': 'All the appetizers and salads were fabulous, the steak was mouth watering and the pasta was delicious!!!', 'term': 'appetizers', 'from': '8', 'to': '18', 'source_dep': [2, 2, 2, 2, 2, 2, 9, 2, 2, 6, 6, 2, 2, 2, 6, 9, 12, 15, 15, 15, 9, 20, 18, 20, 20, 9, 9], 'target_dep': [0, 3, 4, 1, 3, 4, 2, 3, 4, 5, 7, 6, 3, 4, 7, 8, 11, 12, 13, 14, 15, 16, 11, 18, 19, 20, 21], 'edge_type': ['dep', 'sprwrd', 'sprwrd', 'det', 'sprwrd', 'sprwrd', 'nsubj', 'sprwrd', 'sprwrd', 'cc', 'sprwrd', 'conj', 'sprwrd', 'sprwrd', 'sprwrd', 'cop', 'det', 'nsubj', 'cop', 'compound', 'ccomp', 'cc', 'det', 'nsubj', 'cop', 'conj', 'punct'], 'label': 'positive'}


In [4]:
# class Dataset(Dataset):
#     def __init__(self, data_dir, transformation='KW_M', num_classes=3, bert_tokenizer=None, max_length=0, seed=0):
#         random.seed(seed)
#         assert transformation in ['QA_M', 'MLI_M', 'KW_M'], 'Invalid transformation method'
#         assert num_classes in [2, 3], 'Invalid num_classes'
        
#         self.transformation = transformation
#         self.bert_tokenizer = bert_tokenizer
#         self.max_length = max_length
#         self.polarity_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
        
#         # load data
#         self.data = pd.read_pickle(data_dir)
    
#         if num_classes == 2:
#             self.data = [d for d in self.data if d['label'] != 'neutral']
    
#     def transform(self, sample):
#         # Transform input text to 
#         seq1 = sample['text'].lower()
#         term = sample['term'].lower()
        
#         if self.transformation == 'QA_M':
#             seq2 = f'what is the polarity of {term} ?'
#             label = self.polarity_dict[sample['label']]
#         elif self.transformation == 'MLI_M':
#             seq2 = term.lower()
#             label = self.polarity_dict[sample['label']]
#         elif self.transformation == 'KW_M':
#             seq2 = term
#             label = self.polarity_dict[sample['label']]
        
#         return seq1, seq2, label
        
#     def encode_text(self, seq1, seq2):
#         # Encode text for BERT model
#         encoded_text = self.bert_tokenizer.encode_plus(
#             seq1,
#             seq2,
#             add_special_tokens=True,  # Add [CLS] and [SEP]
#             max_length=self.max_length,  # maximum length of a sentence
#             padding='max_length',  # Add [PAD]s
#             truncation=True, # Truncate up to maximum length
#             return_attention_mask=True,  # Generate the attention mask
#             return_tensors='pt',  # Ask the function to return PyTorch tensors
#         )
#         return encoded_text

#     def padding_adj(self, adj):
#         # Padding adj to fixed size
#         pad_size = self.max_length - adj.shape[0]
#         return np.pad(adj, [(0, pad_size), (0, pad_size)], 'constant')
    
#     def get_adj_of_dependency_graph(self, edge_index):
#         edge_reindex = torch.tensor(self.reindex_edge_index(edge_index))
#         dense_adj = to_dense_adj(edge_reindex).squeeze().numpy()
#         return self.padding_adj(dense_adj)
    
#     def reindex_edge_index(self, edge_index):
#         '''
#         Reindex for special token id in BERT
#         '''
#         edge_reindex = [
#             [i+1 for i in edge_index[0]], 
#             [i+1 for i in edge_index[1]],
#         ]
#         return edge_reindex
    
#     def __getitem__(self, item):
#         '''
#         sample = {
#         'id': '813', 
#         'text': 'All the appetizers and salads were fabulous, \\
#         the steak was mouth watering and the pasta was delicious!!!', 
#         'term': 'appetizers', 
#         'from': '8', 
#         'to': '18', 
#         'source_dep': [2, 2, 2, 2, 2, 2, 9, 2, 2, 6, 6, 2, 2, 2, 6, 9, 12, 15, 15, 15, 9, 20, 18, 20, 20, 9, 9], 
#         'target_dep': [0, 3, 4, 1, 3, 4, 2, 3, 4, 5, 7, 6, 3, 4, 7, 8, 11, 12, 13, 14, 15, 16, 11, 18, 19, 20, 21], 
#         'edge_type': ['dep', 'sprwrd', 'sprwrd', 'det', 'sprwrd', 'sprwrd', 'nsubj',\\
#         'sprwrd', 'sprwrd', 'cc', 'sprwrd', 'conj', 'sprwrd', 'sprwrd', 'sprwrd', 'cop',\\
#         'det', 'nsubj', 'cop', 'compound', 'ccomp', 'cc', 'det', 'nsubj', 'cop', 'conj', 'punct'],
#         'label': 'positive'}
#         '''
            
#         # BERT Encoder
#         sample = self.data[item]
#         seq1, seq2, label = self.transform(sample)
#         encoded_text = self.encode_text(seq1, seq2)
#         edge_index = [sample['source_dep'], sample['target_dep']]
#         edge_reindex = self.reindex_edge_index(edge_index)
        
#         single_input = {
#             'id': sample['id'],
#             'text': sample['text'],
#             'seq1': seq1,
#             'seq2': seq2,
#             'term': sample['term'],
#             'label': label, 
#             'input_ids': encoded_text['input_ids'].flatten(),
#             'token_type_ids': encoded_text['token_type_ids'].flatten(),
#             'attention_mask': encoded_text['attention_mask'].flatten(),
#             'edge_index': edge_reindex, 
#         }
#         return single_input

#     def __len__(self):
#         return len(self.data)
    
# class DataModule(pl.LightningDataModule):
#     def __init__(self, params):
#         super().__init__()
#         self.save_hyperparameters(params)

#     def setup(self, stage=None):
#         bert_tokenizer = BertTokenizer.from_pretrained(self.hparams.bert_name)
        
#         # Assign train/val datasets for use in dataloaders
#         if stage == "fit" or stage is None:
#             data_fit = Dataset(
#                 data_dir=self.hparams.data_train_dir,
#                 transformation=self.hparams.transformation,
#                 num_classes=self.hparams.num_classes,
#                 bert_tokenizer=bert_tokenizer,
#                 max_length=self.hparams.max_length,
#                 seed=self.hparams.seed)
            
#             total_samples = data_fit.__len__()
#             train_samples = int(data_fit.__len__() * 0.8)
#             val_samples = total_samples - train_samples
#             self.data_train, self.data_val = random_split(
#                 data_fit, [train_samples, val_samples], generator=torch.Generator().manual_seed(self.hparams.seed))

#         # Assign test dataset for use in dataloader(s)
#         if stage == "test" or stage is None:
#             self.data_test = Dataset(
#                 data_dir=self.hparams.data_test_dir,
#                 transformation=self.hparams.transformation,
#                 num_classes=self.hparams.num_classes,
#                 bert_tokenizer=bert_tokenizer,
#                 max_length=self.hparams.max_length,
#                 seed=self.hparams.seed)
    
#     def reindex_node_by_graph(self, nodes, graph_order):
#         new_nodes = [n + graph_order*self.hparams.max_length for n in nodes]
#         return new_nodes
    
#     def collate_fn(self, batch):        
#         label = torch.tensor([i['label'] for i in batch], dtype=torch.long)
#         input_ids = torch.stack([i['input_ids'] for i in batch])
#         token_type_ids = torch.stack([i['token_type_ids'] for i in batch])
#         attention_mask = torch.stack([i['attention_mask'] for i in batch])
#         source = [self.reindex_node_by_graph(j['edge_index'][0], i) for i,j in enumerate(batch)]
#         target = [self.reindex_node_by_graph(j['edge_index'][1], i) for i,j in enumerate(batch)]
#         edge_index = torch.tensor([
#             [item for sublist in source for item in sublist], 
#             [item for sublist in target for item in sublist], 
#         ], dtype=torch.long)
#         adj = to_dense_adj(edge_index).squeeze() 
#         return {
#             'label': label,
#             'input_ids': input_ids,
#             'token_type_ids': token_type_ids,
#             'attention_mask': attention_mask,
#             'edge_index': edge_index,
#             'adj': adj,
#         }
    
#     def train_dataloader(self):
#         return DataLoader(
#             self.data_train, 
#             batch_size=self.hparams.batch_size, 
#             num_workers=4, 
#             shuffle=False, # Already shuffle in random_split() 
#             drop_last=True, 
#             collate_fn=self.collate_fn,
#         )
    
#     def mytrain_dataloader(self):
#         return DataLoader(
#             self.data_train, 
#             batch_size=self.hparams.batch_size, 
#             num_workers=4, 
#             shuffle=False, # Already shuffle in random_split() 
#             drop_last=False, 
#             collate_fn=self.collate_fn,
#         )
    
#     def val_dataloader(self):
#         return DataLoader(
#             self.data_val, 
#             batch_size=self.hparams.batch_size, 
#             num_workers=4, 
#             shuffle=False,
#             collate_fn=self.collate_fn,
#         )
#     def test_dataloader(self):
#         return DataLoader(
#             self.data_test, 
#             batch_size=self.hparams.batch_size, 
#             num_workers=4, 
#             shuffle=False,
#             collate_fn=self.collate_fn,
#         )

In [5]:
# convert adj to index: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/sparse.html#dense_to_sparse
# write collate_fn for adj

In [2]:
config_file = '../src/config/restaurant_config.json'
config = read_json(config_file)
data_params, model_params = config['data_params'], config['model_params']

In [3]:
data_params = config['data_params']
data = DataModule(data_params)
data.setup()

In [4]:
for i in data.train_dataloader():
    break

## Model

In [2]:
class SentimentClassifier(pl.LightningModule):
    def __init__(self, params):
        super().__init__()
        self.save_hyperparameters(params)
        self.tokenizer = BertTokenizer.from_pretrained(self.hparams.pretrained_bert_name)
        self.bert = BertForSequenceClassification.from_pretrained(
            self.hparams.pretrained_bert_name, num_labels=3, output_hidden_states=True, output_attentions=True, return_dict=False)
        self.hidden_size = self.bert.config.hidden_size
        self.cross_entropy_loss = nn.CrossEntropyLoss()
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer  
    
    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        loss, logits, hidden, _ = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
            labels=labels,
        )
        
        return logits
    
    def margin_loss(self,  embedding_query, embedding_pos, embedding_neg):
        scores_pos = (embeddings_query * embeddings_pos).sum(dim=-1)
        scores_neg = (embeddings_query * embeddings_neg).sum(dim=-1) * self.scale
        return scores_pos - scores_neg
        
    def training_step(self, batch, batch_idx):
        # ['seq1', 'seq2', 'term', 'label', 'input_ids', 'token_type_ids', 'attention_mask']
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
#         acc = utils.calc_accuracy(logits, labels).squeeze()
#         logs = {
#             'loss': ce_loss,
#             'acc': acc,
#         }
#         self.log_dict(logs, prog_bar=True)
        return ce_loss
    
    def validation_step(self, batch, batch_idx):
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
        acc = utils.calc_accuracy(logits, labels).squeeze()
        macro_f1 = utils.calc_f1(logits, labels, avg_type='macro').squeeze()
        micro_f1 = utils.calc_f1(logits, labels, avg_type='micro').squeeze()

        logs = {
            'loss': ce_loss, 
            'acc': acc,
            'macro_f1': macro_f1,
            'micro_f1': micro_f1
        }
        self.log_dict(logs, prog_bar=True)
        return logs
    
    def validation_epoch_end(self, val_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in val_step_outputs]).mean().cpu()
        avg_acc = torch.stack([x['acc'] for x in val_step_outputs]).mean().cpu()
        avg_macro_f1 = torch.stack([x['macro_f1'] for x in val_step_outputs]).mean().cpu()
        avg_micro_f1 = torch.stack([x['micro_f1'] for x in val_step_outputs]).mean().cpu()
        logs = {
            'val_loss': avg_loss, 
            'val_acc': avg_acc,
            'val_macro_f1': avg_macro_f1,
            'val_micro_f1': avg_micro_f1,
        }
        self.log_dict(logs, prog_bar=True)
     
    def test_step(self, batch, batch_idx):
        logits = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['label'],
        )
        
        labels = batch['label']
        ce_loss = self.cross_entropy_loss(logits, labels)        
        acc = utils.calc_accuracy(logits, labels).squeeze()
        macro_f1 = utils.calc_f1(logits, labels, avg_type='macro').squeeze()
        micro_f1 = utils.calc_f1(logits, labels, avg_type='micro').squeeze()

        logs = {
            'loss': ce_loss, 
            'acc': acc,
            'macro_f1': macro_f1,
            'micro_f1': micro_f1
        }
        return logs
    
    def test_epoch_end(self, test_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in test_step_outputs]).mean().cpu()
        avg_acc = torch.stack([x['acc'] for x in test_step_outputs]).mean().cpu()
        avg_macro_f1 = torch.stack([x['macro_f1'] for x in test_step_outputs]).mean().cpu()
        avg_micro_f1 = torch.stack([x['micro_f1'] for x in test_step_outputs]).mean().cpu()

        logs = {
            'test_loss': avg_loss, 
            'test_acc': avg_acc,
            'test_macro_f1': avg_macro_f1,
            'test_micro_f1': avg_micro_f1,
        }
        self.log_dict(logs, prog_bar=True)
        return logs
     

In [12]:
import commentjson
from collections import OrderedDict

def read_json(fname):
    '''
    Read in the json file specified by 'fname'
    '''
    with open(fname, 'rt') as handle:
        return commentjson.load(handle, object_hook=OrderedDict)

def build_model(config):
    data_params, model_params = config['data_params'], config['model_params']
    data = DataModule(data_params)
    model = SynSentimentClassifier(model_params)
    return data, model

In [13]:
def build_trainder(config):
    trainer_params = config['trainer_params']
    data_params = config['data_params']
    
    # callbacks
    checkpoint = ModelCheckpoint(
        dirpath=trainer_params['checkpoint_dir'], 
        filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}-{val_macro_f1:.4f}-{val_micro_f1:.4f}',
        save_top_k=trainer_params['top_k'],
        verbose=True,
        monitor=trainer_params['metric'],
        mode=trainer_params['mode'],
    )
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        min_delta=0.00, 
        patience=trainer_params['patience'],
        verbose=False,
        mode=trainer_params['mode'],
    )
    callbacks = [checkpoint, early_stopping]
    
    # trainer_kwargs
    trainer_kwargs = {
        'max_epochs': trainer_params['max_epochs'],
        'gpus': 1 if torch.cuda.is_available() else 0,
    #     "progress_bar_refresh_rate":p_refresh,
    #     'gradient_clip_val': hyperparameters['grad_clip'],
        'weights_summary': 'full',
        'deterministic': True,
        'callbacks': callbacks,
    }

    trainer = Trainer(**trainer_kwargs)
    return trainer, trainer_kwargs

In [14]:
parser = argparse.ArgumentParser(description='Training.')

parser.add_argument('-config_file', help='config file path', default='../src/config/laptop_config.json', type=str)
parser.add_argument('-f', '--fff', help='a dummy argument to fool ipython', default='1')
args = parser.parse_args()

args.config = read_json(args.config_file)
seed_everything(args.config['data_params']['seed'], workers=True)
data, clf = build_model(args.config)
trainer, trainer_kwargs = build_trainder(args.config)
trainer.fit(clf, data)

Global seed set to 12345
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Che

Validation sanity check: 0it [00:00, ?it/s]

/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/properties.py:301: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_progress_bar_dict()` in `LightingModule`.
  rank_zero_warn(
Global seed set to 12345
/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/properties.py:301: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_progress_bar_dict()` in `LightingModule`.
  rank_zero_warn(
Epoch 10, global step 318: val_loss was not in top 3


Validating: 0it [00:00, ?it/s]

/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/properties.py:301: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_progress_bar_dict()` in `LightingModule`.
  rank_zero_warn(
Epoch 11, global step 347: val_loss was not in top 3


Validating: 0it [00:00, ?it/s]

/home/hoang/miniconda2/envs/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/properties.py:301: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_progress_bar_dict()` in `LightingModule`.
  rank_zero_warn(
Epoch 12, global step 376: val_loss was not in top 3


## Predict

In [37]:
parser = argparse.ArgumentParser(description='Training.')

parser.add_argument('-config_file', help='config file path', default='../src/restaurant_config.json', type=str)
parser.add_argument('-f', '--fff', help='a dummy argument to fool ipython', default='1')
args = parser.parse_args()

args.config = read_json(args.config_file)
seed_everything(args.config['data_params']['seed'], workers=True)
data, clf = build_model(args.config)
trainer, trainer_kwargs = build_trainder(args.config)

Global seed set to 12345
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized fro

In [38]:
paths = sorted(glob.glob('/home/hoang/github/BERT_ABSA/model/restaurants/*.ckpt'))
model_test = SentimentClassifier.load_from_checkpoint(paths[0])
result = trainer.test(model_test, datamodule=data)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Testing: 0it [00:00, ?it/s]

[{'loss': tensor(0.4673, device='cuda:0'), 'acc': tensor(0.8203, dtype=torch.float64), 'macro_f1': tensor(0.7241, dtype=torch.float64), 'micro_f1': tensor(0.8203, dtype=torch.float64)}, {'loss': tensor(0.4060, device='cuda:0'), 'acc': tensor(0.8906, dtype=torch.float64), 'macro_f1': tensor(0.7525, dtype=torch.float64), 'micro_f1': tensor(0.8906, dtype=torch.float64)}, {'loss': tensor(0.4146, device='cuda:0'), 'acc': tensor(0.8359, dtype=torch.float64), 'macro_f1': tensor(0.6356, dtype=torch.float64), 'micro_f1': tensor(0.8359, dtype=torch.float64)}, {'loss': tensor(0.6661, device='cuda:0'), 'acc': tensor(0.7734, dtype=torch.float64), 'macro_f1': tensor(0.6064, dtype=torch.float64), 'micro_f1': tensor(0.7734, dtype=torch.float64)}, {'loss': tensor(0.7138, device='cuda:0'), 'acc': tensor(0.7578, dtype=torch.float64), 'macro_f1': tensor(0.5623, dtype=torch.float64), 'micro_f1': tensor(0.7578, dtype=torch.float64)}, {'loss': tensor(0.7445, device='cuda:0'), 'acc': tensor(0.7500, dtype=torc